In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [3]:
colors = sns.color_palette("deep")

## Load Data

In [2]:
data_path = '../data/Predicting Coupon Redemption/train'
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))
demo_df = pd.read_csv(os.path.join(data_path, 'customer_demographics.csv'))
cmpn_df = pd.read_csv(os.path.join(data_path, 'campaign_data.csv'))
cp_it_df = pd.read_csv(os.path.join(data_path, 'coupon_item_mapping.csv'))
txn_df = pd.read_csv(os.path.join(data_path, 'customer_transaction_data.csv'))
item_df = pd.read_csv(os.path.join(data_path, 'item_data.csv'))

## EDA: Customer Transactions

In [9]:
print(f'total transaction before join: {txn_df.shape[0]}')
txn_df = txn_df.merge(item_df, on='item_id', how='left')
print(f'total transaction after join {txn_df.shape[0]}')

total transaction before join: 1324566
total transaction after join 1324566


In [11]:
txn_df.head(10)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,524,Established,Grocery
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,524,Established,Grocery
5,2012-01-02,1501,57397,1,71.24,-28.14,0.0,524,Established,Grocery
6,2012-01-02,857,12424,1,106.50,-14.25,0.0,971,Established,Grocery
7,2012-01-02,857,14930,1,110.07,0.00,0.0,3235,Established,Meat
8,2012-01-02,857,16657,1,89.05,-35.26,0.0,2011,Established,Packaged Meat
9,2012-01-02,67,10537,3,32.06,0.00,0.0,487,Established,Grocery


In [20]:
print('drop duplicates')
txn_df.drop_duplicates(subset=['date', 'customer_id', 'item_id', 'quantity', 'selling_price', 'other_discount', 'coupon_discount'], inplace=True)
count_dup = txn_df.duplicated(subset=['date', 'customer_id', 'item_id', 'quantity', 'selling_price', 'other_discount', 'coupon_discount']).sum()
print('count duplicates: ', count_dup)

drop duplicates
count duplicates:  0


In [18]:
txn_df['txn_month'] = txn_df['date'].str[:7]
txn_df.groupby('txn_month').count()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
txn_month,,,,,,,,,,
2012-01,12368,12368,12368,12368,12368,12368,12368,12368,12368,12368
2012-02,25037,25037,25037,25037,25037,25037,25037,25037,25037,25037
2012-03,46642,46642,46642,46642,46642,46642,46642,46642,46642,46642
2012-04,68232,68232,68232,68232,68232,68232,68232,68232,68232,68232
2012-05,78929,78929,78929,78929,78929,78929,78929,78929,78929,78929
2012-06,78946,78946,78946,78946,78946,78946,78946,78946,78946,78946
2012-07,80906,80906,80906,80906,80906,80906,80906,80906,80906,80906
2012-08,83861,83861,83861,83861,83861,83861,83861,83861,83861,83861
2012-09,85175,85175,85175,85175,85175,85175,85175,85175,85175,85175


In [21]:
txn_df[txn_df['customer_id'] == 1501]

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,txn_month
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,2012-01
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products,2012-01
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,524,Established,Grocery,2012-01
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery,2012-01
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,524,Established,Grocery,2012-01
...,...,...,...,...,...,...,...,...,...,...,...
1312626,2013-06-29,1501,61261,1,76.94,0.00,0.0,1025,Established,Pharmaceutical,2013-06
1320415,2013-07-02,1501,10044,2,134.64,-128.23,0.0,56,Local,Grocery,2013-07
1320416,2013-07-02,1501,13467,1,95.82,-5.34,0.0,1075,Established,Grocery,2013-07
1320417,2013-07-02,1501,26854,1,35.62,-21.02,0.0,89,Established,Grocery,2013-07
